In [1]:
from IPython.display import display
from ipywidgets import IntSlider, Output
from IPython.display import display
from IPython.display import clear_output
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import urllib.request, urllib.parse, urllib.error
import json
import time
import random
import requests
import sys, time
import pandas as pd 
from pandas.io.json import json_normalize
import pandas as pd 
import numpy as np
import warnings
import ipywidgets as widgets
warnings.filterwarnings('ignore')

In [2]:
#Collecting data and formatting it. 
def getCourseURL():
    courserequested = input(str("Enter the course code: ")).upper()
    url_return = 'https://api.umd.io/v0/courses/'
    url = 'https://api.umd.io/v0/courses/list'
    uh = urllib.request.urlopen(url)
    print('retrieving', url)
    data = uh.read().decode()
    info = json.loads(data)
    #print(json.dumps(info, indent=4, separators=(". ", " = ")))
    courses_array = []
    for courselist in info:
        if courselist['course_id'].startswith(courserequested):
            courses_array.append(courselist['course_id'])
            #print(courselist['course_id'])
            #print(courses_array)
    myString = ",".join(courses_array)
    url_return=url_return+myString
    print(url_return)  
    return url_return

def getCourseURL_ipywidget(courserequested):
    #courserequested = input(str("Enter the course code: ")).upper()
    url_return = 'https://api.umd.io/v0/courses/'
    url = 'https://api.umd.io/v0/courses/list'
    uh = urllib.request.urlopen(url)
    print('retrieving', url)
    data = uh.read().decode()
    info = json.loads(data)
    #print(json.dumps(info, indent=4, separators=(". ", " = ")))
    courses_array = []
    for courselist in info:
        if courselist['course_id'].startswith(courserequested):
            courses_array.append(courselist['course_id'])
            #print(courselist['course_id'])
            #print(courses_array)
    myString = ",".join(courses_array)
    url_return=url_return+myString
    print(url_return)  
    return url_return

def getJSON(url):
    uh = urllib.request.urlopen(url)
    print('retrieving', url)
    data = uh.read().decode()
    info = json.loads(data)
    #making the file so you can read it in your script. 
    with open("coursedata.json", "w") as writeJSON:
        json.dump(json.dumps({"data": info}, indent=3), writeJSON, ensure_ascii=False)
    return (json.dumps({"data": info}, indent=4))
#getJSON(getCourseURL(courserequested))



In [3]:
# UMD.io Data Getting user input for course: 
display(widgets.Output(layout={'border': '2px solid black'}))
display(widgets.HTML(value="<h2>Enter 4 Digit course code/major code for popularity recommendation results:</h2>"))
display(widgets.Output(layout={'border': '2px solid black'}))

df = pd.read_json(getCourseURL(), orient='columns')
#df[['course_id', 'description', 'name']].head(3)  len(df['sections'][1])
df['popularity'] = [len(c) for c in df['sections']] #this assigns a score based on the amount of sections, assumming past popularity merits more sections?!
df['popularity_avg'] = df['popularity']/df['popularity'].mean()
#a = df['grading_method'][1]
#print(len(a))
df.head(3)


A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

Enter the course code: cmsc
retrieving https://api.umd.io/v0/courses/list
https://api.umd.io/v0/courses/CMSC122,CMSC131,CMSC132,CMSC132A,CMSC216,CMSC216H,CMSC250,CMSC250H,CMSC298A,CMSC320,CMSC330,CMSC351,CMSC389N,CMSC390,CMSC396H,CMSC411,CMSC412,CMSC414,CMSC417,CMSC420,CMSC421,CMSC422,CMSC423,CMSC424,CMSC425,CMSC426,CMSC433,CMSC434,CMSC435,CMSC436,CMSC451,CMSC452,CMSC456,CMSC457,CMSC460,CMSC466,CMSC470,CMSC474,CMSC475,CMSC498A,CMSC498L,CMSC498O,CMSC498U,CMSC499A,CMSC631,CMSC642,CMSC644,CMSC661,CMSC664,CMSC712,CMSC722,CMSC726,CMSC733,CMSC740,CMSC764,CMSC773,CMSC798,CMSC798F,CMSC799,CMSC818N,CMSC818W,CMSC818X,CMSC818Y,CMSC828B,CMSC828F,CMSC828U,CMSC828X,CMSC838J,CMSC858M,CMSC898,CMSC899


,core,course_id,credits,department,dept_id,description,gen_ed,grading_method,name,relationships,sections,semester,popularity,popularity_avg
0,[I],CMSC122,3,Computer Science,CMSC,Introduction to computer programming in the co...,[DSSP],[Regular],Introduction to Computer Programming via the Web,"{'prereqs': None, 'coreqs': None, 'restriction...","[CMSC122-0101, CMSC122-0201]",201901,2,0.830409
1,[],CMSC131,4,Computer Science,CMSC,Introduction to programming and computer scien...,[],[Regular],Object-Oriented Programming I,"{'prereqs': None, 'coreqs': 'MATH140; and perm...","[CMSC131-0101, CMSC131-0102, CMSC131-0103, CMS...",201901,20,8.304094
2,[],CMSC132,4,Computer Science,CMSC,Introduction to use of computers to solve prob...,[],[Regular],Object-Oriented Programming II,{'prereqs': 'Minimum grade of C- in CMSC131; o...,"[CMSC132-0101, CMSC132-0102, CMSC132-0103, CMS...",201901,23,9.549708


In [4]:
#Getting final version of what data i need and calulating the mean.  
#df[['course_id', 'description', 'name', 'rank']].head(15).to_csv('test.csv', sep='\t', encoding='utf-8')
df[['course_id', 'description', 'name', 'popularity', 'popularity_avg']].head(15)
#weighted rating of courses
C = df['popularity'].mean()
print("weighted rating of popularity (sectionCount): " + str(C)) # section count is popularity
# Calculate the minimum number of votes required to be in the chart, m
m = df['popularity'].quantile(0.92)
print("minimum number of rank required to be in the chart: " + str(m))
# Filter out all qualified classes into a new DataFrame
q_classes = df.copy().loc[df['popularity'] >= m]
q_classes.shape
#mimicing their catagorical classifier
def weighted_rating(x, m=m, C=C):
    v = x['popularity']
    R = x['popularity_avg']
    # Calculation based on the IMDB formula 
    return (v/(v+m) * R) + (m/(m+v) * C) # thing to modify for courses?
#adding new column with score
q_classes['score'] = q_classes.apply(weighted_rating, axis=1)
#sorting movvies based on ^ and printing
q_classes = q_classes.sort_values('score', ascending=False)
q_classes[['course_id', 'description', 'name', 'popularity', 'popularity_avg']].head(15)


weighted rating of popularity (sectionCount): 2.408450704225352
minimum number of rank required to be in the chart: 3.4000000000000057


,course_id,description,name,popularity,popularity_avg
2,CMSC132,Introduction to use of computers to solve prob...,Object-Oriented Programming II,23,9.549708
1,CMSC131,Introduction to programming and computer scien...,Object-Oriented Programming I,20,8.304094
10,CMSC330,The semantics of programming languages and the...,Organization of Programming Languages,20,8.304094
6,CMSC250,Fundamental mathematical concepts related to c...,Discrete Structures,18,7.473684
4,CMSC216,Machine representation of data including integ...,Introduction to Computer Systems,16,6.643275
11,CMSC351,A systematic study of the complexity of some e...,Algorithms,4,1.660819


In [5]:
#Content based Recommendations from Courses: 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
display(widgets.Output(layout={'border': '2px solid black'}))
display(widgets.HTML(value="<h2>Content based Recommendations from Courses Descriptions:</h2>"))
display(widgets.Output(layout={'border': '2px solid black'}))

majorCode = widgets.HTML(
    value="<h3>Enter 4 Digit course code/Major code</h3>")
#buttonMajor = widgets.Button(description="Get Major")
#majorText = widgets.Text()
#display(majorCode)
#display(majorText)
#display(buttonMajor)


def buttonMajor_clicked(b):
    m = majorText.value
    majortextFormatted = m.upper().strip()
    display('Current Major: ' + majortextFormatted)
    df = pd.read_json(getCourseURL_ipywidget(majortextFormatted), orient='columns')
    
#buttonMajor.on_click(buttonMajor_clicked)

#Content based Recommendations from Courses: 
df = pd.read_json(getCourseURL(), orient='columns')
df['description'].head()

#tfidf = TfidfVectorizer(stop_words) #stop words like in R!
tfidf = TfidfVectorizer(stop_words='english',max_features=650) #stop words like in R!
df['description'] = df['description'].fillna('') 

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df['description'])
#Output the shape of tfidf_matrix
tfidf_matrix.shape

from sklearn.metrics.pairwise import linear_kernel
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

indices = pd.Series(df.index, index=df['course_id']).drop_duplicates()
def get_recommendations(courseName, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[courseName]

    # Get the pairwsie similarity scores of all
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the courses based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 5 most similar classes
    sim_scores = sim_scores[1:6]

    # Get the classes indices
    courses_indices = [i[0] for i in sim_scores]

    # Return the top 6 most similar classes
    return df[['course_id', 'description']].iloc[courses_indices]


A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

Enter the course code: gvpt
retrieving https://api.umd.io/v0/courses/list
https://api.umd.io/v0/courses/GVPT170,GVPT200,GVPT201,GVPT202,GVPT203,GVPT221,GVPT241,GVPT273,GVPT282,GVPT301,GVPT306,GVPT317,GVPT319C,GVPT319D,GVPT319E,GVPT319G,GVPT331,GVPT355,GVPT356,GVPT359F,GVPT388,GVPT388L,GVPT388M,GVPT388W,GVPT390,GVPT396,GVPT397,GVPT409A,GVPT409G,GVPT409H,GVPT409O,GVPT409W,GVPT420,GVPT423H,GVPT429E,GVPT432,GVPT439A,GVPT439D,GVPT439F,GVPT445,GVPT449G,GVPT449I,GVPT456,GVPT459E,GVPT459H,GVPT459M,GVPT459N,GVPT459P,GVPT459R,GVPT459V,GVPT461,GVPT474,GVPT475,GVPT477,GVPT722,GVPT743,GVPT771,GVPT799,GVPT803,GVPT808B,GVPT808C,GVPT828B,GVPT831,GVPT834,GVPT888P,GVPT898,GVPT899


In [6]:
from sklearn.metrics.pairwise import linear_kernel
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

indices = pd.Series(df.index, index=df['course_id']).drop_duplicates()
def get_recommendations(courseName, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[courseName]

    # Get the pairwsie similarity scores of all
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the courses based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 5 most similar classes
    sim_scores = sim_scores[1:6]

    # Get the classes indices
    courses_indices = [i[0] for i in sim_scores]

    # Return the top 6 most similar classes
    return df[['course_id', 'description']].iloc[courses_indices]


#courseName = input(str('Enter course code and course level nummber: ')).upper().strip()
pd.set_option('display.max_colwidth', -1)
#get_recommendations(courseName)

In [7]:
display(widgets.Output(layout={'border': '2px solid black'}))
buttonCourse = widgets.Button(description="Recommend")
clear = widgets.Button(description="Clear")
text = widgets.Text()
out = widgets.Output()
markup = widgets.HTML(
    value="<h3>Enter course number for "+df['course_id'][1][:-3]+" classes:</h3>")

display(markup)
display(text)
display(buttonCourse, clear)

display(out)


def on_button_clicked(b):
    with out:
        try:
            textOG = text.value
            textCapped = textOG.upper().strip()
            display(widgets.Output(layout={'border': '2px solid black'}))
            print("Course: " + textCapped)
            display(get_recommendations(textCapped))
            display(widgets.Output(layout={'border': '2px solid black'}))
        except:
            print('Make sure the course code is valid \nTt must match the major code above of: '+df['course_id'][1][:-3])
            print('Refresh to enter a new code. Annoying, I know..')
        
display(widgets.Output(layout={'border': '2px solid black'}))

    
def clearalloutput(b):
    out.clear_output()
    
    
buttonCourse.on_click(on_button_clicked)
clear.on_click(clearalloutput)


A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget